# Nonlinear Regression of the SUPCRTBL database

The SUPCRTBL database was created by members of the Zhu Laboratory at IUB to aid in geochemical modeling of aqueous minerals, in particular Rare Earth Elements (REEs)

An excel copy of the database has been provided, file name "spronsbl_aqeousREE".



Shock, E. L., Sassani, D. C., Willis, M., & Sverjensky, D. A. (1997). Inorganic species in geologic fluids: correlations among standard molal thermodynamic properties of aqueous ions and hydroxide complexes. Geochimica et Cosmochimica Acta, 61(5), 907-950.

In [1]:
"""import libraries"""
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy
import xarray as xr
import torch
import torch.nn as nn


In [6]:
""" Read the tmin data file. """
filename = r"C:\Users\janel\OneDrive\Documents\IU grad\Classes F23\EAS 690 Advanced Data Science\G690_Test\research_project\spronsbl_aqueousREE.xlsx"

# read in the data file, 
supcrtbl = pd.read_excel(filename, sheet_name = "core", header = 0, skiprows = 0)

# show the top of the dataset
supcrtbl.head()

,supcrtbl_aqueous_tb_id,Data_source,Species_type,Name,Structural_formula,Abbreviation,Chemical_formula,Reference,Date,"Gibbs_free_energy_of_formation (DGf0, kJ/mol)","Enthalpy_of_formation (DHf0, kJ/mol)","Entropy (S0, J/mol/K)",HKF_coefficient_a1 (10^1 J/mol/bar),HKF_coefficient_a2 (10^-2 J/mol),HKF_coefficient_a3 (J*K/mol/bar),HKF_coefficient_a4 (10^-4 J*K/mol),HKF_coefficient_c1 (J/mol/K),HKF_coefficient_c2 (10^-4 J*K/mol),"Born_coefficient (omega, 10^-5 J/mol)",Charge
0,1,supcrtbl.dat,Type 6,Ag(CO3)-,Ag(CO3)-,Ag(CO3)-,Ag(1)C(1)O(3)-(1),SSH97,97.09.29,-466.223,-596.178,0.000,9.4508,-9.4692,27.7508,-11.2357,63.6583,-21.3899,6.8241,-1
1,2,supcrtbl.dat,Type 6,Ag(CO3)2-3,Ag(CO3)2-3,Ag(CO3)2-3,Ag(1)C(2)O(6)-(3),SSH97,97.09.29,-991.148,-1272.773,-79.496,15.5344,5.3856,21.9120,-11.8499,152.9624,-36.8163,21.3351,-3
2,3,supcrtbl.dat,Type 6,Ag+,Ag+,Ag+,Ag(1)+(1),SSWS97,97.11.07,77.099,105.751,73.387,7.2320,-14.8984,29.9139,-11.0115,53.4975,-5.9639,0.9037,1
3,4,supcrtbl.dat,Type 6,Ag+2,Ag+2,Ag+2,Ag(1)+(2),SH88,88.02.16,269.031,268.613,-87.864,-1.7987,-36.9489,38.5807,-10.0998,63.6905,-17.2138,5.5233,2
4,5,supcrtbl.dat,Type 6,"AgCl,aq",AgCl,"AgCl,aq",Ag(1)Cl(1)+(0),SSH97,97.09.16,-73.011,-76.442,142.674,21.7936,20.6685,15.9055,-12.4817,41.0735,-6.9864,-0.1255,0


In [ ]:
class Net(nn.Module):

    def __init__(self, n_relus : int = 2): #int tels py that n_relus is an integer w default val 2
        #iniitalize the parent class
        super().__init__()

        #create a neural network
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(1, n_relus), #layer 1 #n_relus is our output, which is defined as having 2 parts above
            nn.ReLU(), #layer 2
            nn.Linear(n_relus, 1), #layer 3 #both a linear operation on inputs and a sum

        )
    
    def forward(self, x):
        """Define the forward pass; how the network should act like a funciton"""
        out = self.linear_relu_stack(x)
        return  out